<a href="https://colab.research.google.com/github/DaryaTereshchenko/ukr/blob/main/SM_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
news_0 = pd.read_csv("https://github.com/DaryaTereshchenko/ukr/blob/main/data/news_0.csv",  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_0['class'] = 0

news_1 = pd.read_csv("https://github.com/DaryaTereshchenko/ukr/blob/main/data/news_1.csv",  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_1['class'] = 1

news_2 = pd.read_csv(https://github.com/DaryaTereshchenko/ukr/blob/main/data/news_2.csv,  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_2['class'] = 2

news_3 = pd.read_csv("https://github.com/DaryaTereshchenko/ukr/blob/main/data/news_3.csv",  sep=";", decimal=",", 
                   skiprows=[1,2,3,4], low_memory=False, on_bad_lines='skip').sort_values(by=["CODE"])
news_3['class'] = 3

In [ ]:
frames = [news_0, news_1, news_2, news_3]
data = pd.concat(frames).drop(["CODE"], axis=1)
display(data)

In [ ]:
X = data.drop(["class"], axis=1)
y = data['class'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.1, random_state=66)

In [ ]:
# Normalize the data
sc = StandardScaler()
normed_train_data = pd.DataFrame(sc.fit_transform(X_train), columns = X.columns)
normed_test_data = pd.DataFrame(sc.fit_transform(X_test), columns = X.columns)
normed_val_data = pd.DataFrame(sc.fit_transform(X_val), columns = X.columns)

In [ ]:
# Encode the categories
encoder = OneHotEncoder()
encoder.fit(y_train.reshape(-1, 1))
train_l = encoder.transform(y_train.reshape(-1, 1)).toarray()

In [ ]:
encoder.fit(y_val.reshape(-1, 1))
val_l = encoder.transform(y_val.reshape(-1, 1)).toarray()

In [ ]:
encoder.fit(y_test.reshape(-1, 1))
test_l = encoder.transform(y_test.reshape(-1, 1)).toarray()

In [ ]:
def build_model():
    x = Input(shape=(101), dtype=tf.float32)
    
    x = Flatten()(x)
    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=x, outputs=output)

    model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"])
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
model.fit(normed_train_data, train_l, batch_size = 16, epochs = 10, verbose=1, validation_data=(normed_val_data, val_l))

In [ ]:
y_pred = model.predict(normed_test_data).argmax(axis=1)
encoder.fit(y_pred.reshape(-1, 1))
y_pred_one_hot = encoder.transform(y_pred.reshape(-1, 1)).toarray()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print("=== Classification Report ===")
print(classification_report(test_l, y_pred_one_hot))
print('\n')
print("=== Accuracy Score ===")
print(accuracy_score(test_l, y_pred_one_hot))

In [ ]:
y_test = encoder.inverse_transform(test_l).flatten()
y_pred = encoder.inverse_transform(y_pred_one_hot).flatten()

In [ ]:
def plot_cm(y_true, y_pred, normalize,
    labels=None,
    title="",
    cmap=plt.cm.Blues,
):
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=cmap)
    plt.title(title)
    plt.show()
plot_cm(y_test, y_pred, "true")